In [ ]:
# to print all output for a cell instead of only last one 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# IMPORTS 

from pathlib import Path 
import os 

import pandas as pd
import numpy as np

In [ ]:
# PATHS 

DATA_FOLDER = Path(*Path().absolute().parts[:-1]) / 'data' # directory containing the notebook

Load data


In [ ]:
import json 

json_file_path_train = DATA_FOLDER / 'Twibot-20/train.json'
json_file_path_val = DATA_FOLDER / 'Twibot-20/dev.json'
json_file_path_test = DATA_FOLDER / 'Twibot-20/test.json'

with open(json_file_path_train, 'r') as tr:
     contents = json.loads(tr.read())
     train_df = pd.json_normalize(contents)
     train_df['split'] = 'train'

with open(json_file_path_val, 'r') as vl:
     contents = json.loads(vl.read())
     val_df = pd.json_normalize(contents) 
     val_df['split'] = 'val'

with open(json_file_path_test, 'r') as ts:
     contents = json.loads(ts.read())
     test_df = pd.json_normalize(contents) 
     test_df['split'] = 'test'

df = pd.concat([train_df,val_df,test_df],ignore_index=True) # merge three datasets
df.dropna(subset=['tweet'], inplace=True)  # remove rows withot any tweet 
df.set_index(keys='ID',inplace=True) # reset index

# split dataframe in two : tweet and account data 
tweets_df = df[['tweet','label','split']].reset_index()
tweets_df = tweets_df.explode('tweet').reset_index(drop=True)
tweets_df.rename(columns={"ID": "account_id"}, inplace=True)

account_df = df.drop('tweet',axis=1).reset_index()
account_df.rename(columns={"ID": "account_id"}, inplace=True)


DATA PROCESSING AND CLEANING  

In [ ]:
import emoji
from nltk.tokenize import TweetTokenizer

from ttp import ttp 
parser = ttp.Parser(include_spans=True)

from emot.core import emot
emot_obj = emot()

tk = TweetTokenizer(reduce_len=True,preserve_case=False)

CASHTAG = "(?<!\S)\$[A-Z]+(?:\.[A-Z]+)?(?!\S)"   # to check  (?:\.[A-Z]+)?
EMAIL = r"""[\w.+-]+@[\w-]+\.(?:[\w-]\.?)+[\w-]"""
MONEY = "[$£][0-9]+(?:[.,]\d+)?[k+]?|[0-9]+(?:[.,]\d+)?[k+]?[$£]"  
SPECIAL_NUMBERS = r"""(?:[+\-]?\d+[,/.:-]\d+[+\-]?)""" #'(?:[+\-]?\d+[,/.:-]\d+[+\-]?)'
HASHTAG = r"""(?:\#+[\w_]+[\w\'_\-]*[\w_]+)"""
HANDLES = r"""(?:@[\w_]+)""" 


TO_REPLACE = [CASHTAG,EMAIL,MONEY,SPECIAL_NUMBERS,HASHTAG,HANDLES]
REPLACE_WITH = ['<stock>','<email>','<money>','<number>','<hashtag>','<username>']

# tweets_df['tweet'].replace(TO_REPLACE,REPLACE_WITH,regex=True,inplace=True)

def replace_links(sentence: str):
        result = parser.parse(sentence, html=False)
        urls = dict(result.urls).keys()
        for url in urls:
                sentence = sentence.replace(url,'<url>')
        return sentence 

def replace_emoticons(sentence : str):
        emoticons = emot_obj.emoticons(sentence)
        for emoticon in emoticons['value']:
                sentence = sentence.replace(emoticon,'<emoticon>')
                
        return sentence

def last_process(sentence : str):
        sentence = emoji.replace_emoji(sentence,'<emoji>')
        sentence = tk.tokenize(sentence)
        

# tweets_df['tweet'] = tweets_df['tweet'].apply(replace_links)
# tweets_df['tweet'] = tweets_df['tweet'].apply(last_process)


In [ ]:
tweets_df2 = tweets_df.copy(deep=True)

In [ ]:
tweets_df2['tweet'] = tweets_df2['tweet'].apply(replace_links)
tweets_df2['tweet'] = tweets_df2['tweet'].apply(replace_emoticons)

In [ ]:
print(tweets_df.loc[962,'tweet'])
print(tweets_df2.loc[962,'tweet'])

In [ ]:
from ttp import ttp 
parser = ttp.Parser(include_spans=True)

t : str = tweets_df.loc[1, 'tweet']
t

result = parser.parse(t, html=False)
spans = dict(result.urls).keys()
spans
for span in spans:
    t = t.replace(span,'<url>')

t
sentence = tk.tokenize(t)
print(sentence)

In [ ]:
from emot.core import emot
emot_obj = emot()

emoticons = emot_obj.emoticons(tweets_df.loc[1,'tweet'])
for emoticon in emoticons['value']:
    print(emoticon)

In [ ]:
prova = tweets_df['tweet'].replace([MONEY,NUMBER],['<MONEY>','<NUMBER>'],regex=True)

In [ ]:
df2 = tweets_df[tweets_df['tweet'].str.contains(':\)', regex=True)]
df2



l = [':\)','\(:',':-\)','\s:\^\)\s','\s=\]\s','\s:-\(\s','\s:\(\s','\s:\(\s','\^_\^','\s:\'\(\s','\s:-D\s',]

In [ ]:
print(tweets_df.loc[962,'tweet'])
print(prova.loc[1999693])




In [ ]:
import emoji
s = emoji.replace_emoji(tweets_df.loc[1998135,'tweet'],'<EMOJI>')
s

In [ ]:
from emot.core import emot
emot_obj = emot()

emoticons = emot_obj.bulk_emoticons(tweets_df.tweet.to_list())
emoji = emot_obj.bulk_emoji(tweets_df.tweet.to_list())


In [ ]:
emoticons[1999110]
emoji[1999110]

In [ ]:
import gensim
import gensim.downloader as gloader
from gensim.models import KeyedVectors

glove_model_cached_path = DATA_FOLDER / 'glove_vectors.txt'
glove_model_download_path = 'glove-twitter-200'

def load_glove_emb(force_download = False):   
    """
        Download the glove embedding model and returns it 
    """
    emb_model = None

    if os.path.exists(glove_model_cached_path) and not force_download: 
        print('found cached glove vectors in data folder, retrieving the file...')
        emb_model = KeyedVectors.load_word2vec_format(glove_model_cached_path, binary=True)
        print('vectors loaded')

    else:
        print('downloading glove embeddings...')        
        emb_model = gloader.load(glove_model_download_path)

        print('saving glove embeddings to file')  
        emb_model.save_word2vec_format(glove_model_cached_path, binary=True)
        
    return emb_model

force_download = False      # to download glove model even if the vectors model has been already stored. Mainly for testing purposes

glove_embeddings = load_glove_emb(force_download)

In [ ]:
glove_embeddings.most_similar("stock")

In [ ]:
tweets_df.iloc[369].tweet


In [ ]:
import nltk
import emoji
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import string 

nltk.download('wordnet', quiet=True) 
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

def preprocess_pipeline(sentence:str):
    """Apply standard preprocessing"""

    #put everything to lowercase
    sentence = sentence.lower()

    #remove all unnecessary spaces and return a list of words
    sentence = sentence.split()

    return sentence

def preprocess_type3(sentence:str):
    """Lower, split, url to 'uuuuu', account to 'ttttt', hashtag to 'ggggg', emoji to 'eeeee', translate to UNICODE, no punctuation"""

    sentence = preprocess_pipeline(sentence)

    #replace url link string with 'uuuuu'
    sentence = ['uuuuu' if 'http' in word else word for word in sentence]

    #replace account link string with 'ttttt'
    sentence = ['ttttt' if '@' in word else word for word in sentence]

    #replace hashtag with 'ggggg'
    sentence = ['ggggg' if '#' in word else word for word in sentence]

    #replace emoticons with 'eeeee'
    sentence = ['eeeee' if emoji.is_emoji(word) else word for word in sentence]
    
    #transliterates any UNICODE string into the closest possible representation in ASCII text
    # sentence = [unidecode.unidecode(word) for word in sentence]

    #remove all punctuation
    sentence = [word.translate(str.maketrans(dict.fromkeys(string.punctuation,''))) for word in sentence]

    #if a word has one non-ASCII character: remove
    sentence = [word for word in sentence if word.isascii()]

    sentence = [word for word in sentence if word != '']

    return sentence

from nltk.tokenize import TweetTokenizer

tk = TweetTokenizer(reduce_len=True,preserve_case=False)

In [ ]:
print(f"Processed tweet with twTk:\n{tk.tokenize('<unk> good')}")

emoji. is_emoji(':)')


In [ ]:

from nltk.tokenize import TweetTokenizer

tk = TweetTokenizer(reduce_len=True,preserve_case=False)

tweet = tweets_df.loc[602396, 'tweet']   #1239
print(f"Original tweet:\n{tweet}")
print(f"Processed tweet with twTk:\n{tk.tokenize(tweet)}")
print(f"Processed tweet with type3:\n{preprocess_type3(tweet)}")

